In [2]:
import pandas as pd
import numpy as np
import geopandas as gp
import adjustText as aT
import matplotlib.pylab as plt
from sklearn import neighbors, datasets
from matplotlib.colors import ListedColormap

df_train = pd.read_csv('data/train_filtrado.csv')
df_test = pd.read_csv('data/test.csv')

In [ ]:
maps = gp.read_file('data/maps/dest_2010gw.shp')

In [ ]:
vals = df_train[(~df_train.lat.isnull()) & (~df_train.lng.isnull()) & (df_train.lat < 32.769750) & (df_train.lat > 14) & (df_train.lng > -118) & (df_train.lng < -85)]
ax = maps.plot(color='white', edgecolor='black')
vals.plot.scatter('lng', 'lat', ax=ax, c='precio', colormap='jet', figsize=(30, 15), colorbar=False,
                title='Precio de propiedad y su ubicación en el mapa - México');

sm = plt.cm.ScalarMappable(cmap='jet', 
                           norm=plt.Normalize(vmin=vals.precio.min(),
                                              vmax=vals.precio.max()))
plt.colorbar(sm, label='precio');

In [ ]:
vals.shape

In [5]:
def merge_ciudades(df):
    ciudades_centricas = pd.read_csv('./data/ciudadcentrica_por_provincia.csv')
    df_centro = pd.merge(df, ciudades_centricas, on = 'ciudad', how= 'right').rename(columns={'provincia_x': 'provincia'})
    df_centro['centro'] =  df_centro['centro'].fillna(False)
    df_centro_promedio = df_centro.groupby('provincia')[['lat', 'lng']].agg('mean').rename(columns = {'lat': 'lat_promedio', 'lng': 'lng_promedio'}).reset_index()
    return df_centro_promedio

In [6]:
#Verifico que todo se encuentre en rango:
#df_centro_promedio[(~df_centro_promedio.lat_promedio.isnull()) & (~df_centro_promedio.lng_promedio.isnull()) & (df_centro_promedio.lat_promedio < 32.769750) & (df_centro_promedio.lat_promedio > 14) & (df_centro_promedio.lng_promedio > -118) & (df_centro_promedio.lng_promedio < -85)].shape

In [7]:
def feature_distancias(df, df_fill=None):
    if df_fill is None:
        df_fill = df
    
    
    df_centro_promedio = merge_ciudades(df_fill)
    df_merge = pd.merge(df, df_centro_promedio, on = 'provincia', how= 'left')
    df['distancia_ciudad_centrica'] = np.sqrt((df_merge.lat - df_merge.lat_promedio)**2 + (df_merge.lng -df_merge.lng_promedio)**2)
    df['distancia_ciudad_centrica'] = df['distancia_ciudad_centrica'].fillna(df['distancia_ciudad_centrica'].mean())
    
    return df